# Ex - GroupBy

### Introduction:

GroupBy can be summarized as Split-Apply-Combine.

Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

Check out this [Diagram](http://i.imgur.com/yjNkiwL.png)  
### Step 1. Import the necessary libraries

In [38]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz

!pip install -q findspark



In [39]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

import findspark
findspark.init()

import pandas as pd

from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

import pyspark.sql.functions as F

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/drinks.csv). 

In [40]:
url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/drinks.csv'

### Step 3. Assign it to a variable called drinks.

In [41]:
from pyspark import SparkFiles
spark.sparkContext.addFile(url)
drinks=spark.read.csv(SparkFiles.get("drinks.csv"), header=True)

### Step 4. Which continent drinks more beer on average?

In [12]:
drinks.show()

+-----------------+-------------+---------------+-------------+----------------------------+---------+
|          country|beer_servings|spirit_servings|wine_servings|total_litres_of_pure_alcohol|continent|
+-----------------+-------------+---------------+-------------+----------------------------+---------+
|      Afghanistan|            0|              0|            0|                         0.0|       AS|
|          Albania|           89|            132|           54|                         4.9|       EU|
|          Algeria|           25|              0|           14|                         0.7|       AF|
|          Andorra|          245|            138|          312|                        12.4|       EU|
|           Angola|          217|             57|           45|                         5.9|       AF|
|Antigua & Barbuda|          102|            128|           45|                         4.9|       NA|
|        Argentina|          193|             25|          221|          

In [19]:
drinks.groupby("continent").agg(F.avg(F.col("beer_servings")).alias("avg_beers")).sort(F.col("avg_beers"), ascending=False).show()

+---------+------------------+
|continent|         avg_beers|
+---------+------------------+
|       EU|193.77777777777777|
|       SA|175.08333333333334|
|       NA|145.43478260869566|
|       OC|           89.6875|
|       AF|61.471698113207545|
|       AS| 37.04545454545455|
+---------+------------------+



### Step 5. For each continent print the statistics for wine consumption.

In [32]:
from pyspark.sql import DataFrameStatFunctions as statFunc

In [47]:
drinks.groupby("continent").agg(F.sum("wine_servings"), F.avg("wine_servings"), F.mean("wine_servings")).show()

+---------+------------------+------------------+------------------+
|continent|sum(wine_servings)|avg(wine_servings)|avg(wine_servings)|
+---------+------------------+------------------+------------------+
|       NA|             564.0| 24.52173913043478| 24.52173913043478|
|       SA|             749.0|62.416666666666664|62.416666666666664|
|       AS|             399.0| 9.068181818181818| 9.068181818181818|
|       OC|             570.0|            35.625|            35.625|
|       EU|            6400.0|142.22222222222223|142.22222222222223|
|       AF|             862.0|16.264150943396228|16.264150943396228|
+---------+------------------+------------------+------------------+



### Step 6. Print the mean alcohol consumption per continent for every column

In [56]:
funs = [F.mean]
aggregate = [col for col in drinks.columns if "servings" in col]
exprs = [f(F.col(c)) for f in funs for c in aggregate]
drinks.groupby("continent").agg(*exprs).show()

+---------+------------------+--------------------+------------------+
|continent|avg(beer_servings)|avg(spirit_servings)|avg(wine_servings)|
+---------+------------------+--------------------+------------------+
|       NA|145.43478260869566|   165.7391304347826| 24.52173913043478|
|       SA|175.08333333333334|              114.75|62.416666666666664|
|       AS| 37.04545454545455|   60.84090909090909| 9.068181818181818|
|       OC|           89.6875|             58.4375|            35.625|
|       EU|193.77777777777777|  132.55555555555554|142.22222222222223|
|       AF|61.471698113207545|  16.339622641509433|16.264150943396228|
+---------+------------------+--------------------+------------------+



### Step 7. Print the median alcohol consumption per continent for every column

### Step 8. Print the mean, min and max values for spirit consumption.
#### This time output a DataFrame

In [57]:
funs = [F.mean, F.min, F.max]
aggregate = [col for col in drinks.columns if "servings" in col]
exprs = [f(F.col(c)) for f in funs for c in aggregate]
drinks.groupby("continent").agg(*exprs).show() 

+---------+------------------+--------------------+------------------+------------------+--------------------+------------------+------------------+--------------------+------------------+
|continent|avg(beer_servings)|avg(spirit_servings)|avg(wine_servings)|min(beer_servings)|min(spirit_servings)|min(wine_servings)|max(beer_servings)|max(spirit_servings)|max(wine_servings)|
+---------+------------------+--------------------+------------------+------------------+--------------------+------------------+------------------+--------------------+------------------+
|       AF|61.471698113207545|  16.339622641509433|16.264150943396228|                 0|                   0|                 0|                98|                  98|                 9|
|       AS| 37.04545454545455|   60.84090909090909| 9.068181818181818|                 0|                   0|                 0|                99|                  97|                 9|
|       EU|193.77777777777777|  132.55555555555554|142.